In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import tqdm
torch.manual_seed(42)

import warnings
warnings.filterwarnings('ignore')
print(torch.__version__)

In [ ]:
Device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Not using Transform in getitem, as data is very small we will store modified data in class object and will do all modification at once while initializing
# Normally it is optimal and efficient to do these modifications while getting the items (in __getitem__) from the dataset - this maintain original data in object init and only 
# do operations only on calling the batch.
class spaceDataset(Dataset):
    def __init__(self, csv_file, target_col = None, drop_col: list = None, split_col: str = None, dummy_col: list = None, norm_col: list = None, scaler = None, Device = 'cpu'):
        self.dataframe = pd.read_csv(csv_file)
        self.drop_col = drop_col
        self.split_col = split_col
        self.dummy_col = dummy_col
        self.target_col = target_col
        self.norm_col = norm_col
        self.scaler  = scaler
        self.Device = Device
        
        if self.drop_col:
            self.dataframe.drop(columns=self.drop_col, inplace=True)
            
        if self.target_col:
            self.dataframe.dropna(inplace=True)
        else:
            self.dataframe.fillna(method='ffill', inplace = True)  # Need to put some logic in this, can go with 0 for the base model as the test nan values count is very low.
        
        if self.split_col:
            self.dataframe[[self.split_col+'_0', self.split_col+'_1', self.split_col+'_2']] = self.dataframe[self.split_col].str.split('/', expand=True)
            self.dataframe.drop(columns=[self.split_col], inplace=True)
        
        if self.dummy_col:
            self.dataframe = pd.get_dummies(self.dataframe, columns = self.dummy_col, drop_first = True)
        self.dataframe = self.dataframe.astype('float32')
        
        if self.scaler and self.target_col:
            self.scaler.fit(self.dataframe[self.norm_col])
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx: int):
        row = self.dataframe.iloc[idx]
        row[self.norm_col] = self.scaler.transform([row[self.norm_col]])[0]
        if self.target_col:
            target = row[self.target_col]
            features = row.drop(self.target_col)
            
            # Convert features and target to tensors
            features_tensor = torch.tensor(features.values, dtype=torch.float32, device = self.Device)
            target_tensor = torch.tensor(target, dtype=torch.float32, device = self.Device)

            return features_tensor, target_tensor
        return torch.tensor(row.values, dtype = torch.float32, device = self.Device)

In [ ]:
class BaseClassifier(nn.Module):
    def __init__(self, input_size = None):
        super().__init__()
        # Where we define all the parts of the model
        self.linear_1 = nn.Linear(input_size, 42)
        self.bnorm_1 = nn.BatchNorm1d(42)
        self.relu_1 = nn.LeakyReLU(negative_slope=0.03)
        self.linear_2 = nn.Linear(42, 84)
        self.bnorm_2 = nn.BatchNorm1d(84)
        self.relu_2 = nn.LeakyReLU(negative_slope=0.03)
        self.linear_3 = nn.Linear(84, 128)
        self.bnorm_3 = nn.BatchNorm1d(128)
        self.relu_3 = nn.LeakyReLU(negative_slope=0.03)
        self.linear_4 = nn.Linear(128, 1)
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.linear_1(x)
        x = self.bnorm_1(x)
        x = self.relu_1(x)
        x = self.linear_2(x)
        x = self.bnorm_2(x)
        x = self.relu_2(x)
        x = self.linear_3(x)
        x = self.bnorm_3(x)
        x = self.relu_3(x)
        output = self.linear_4(x)
        return output

In [ ]:
csv_file = '/kaggle/input/spaceship-titanic/train.csv'
target_col = 'Transported'
drop_col = ['PassengerId', 'Name']
split_col = 'Cabin'
dummy_col = ['HomePlanet', 'Destination', split_col+'_0', split_col+'_2']
norm_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', split_col+'_1']
scaler = StandardScaler()

In [ ]:
dataset = spaceDataset(csv_file, target_col, drop_col, split_col, dummy_col, norm_col, scaler, Device)
# Define split sizes
total_size = len(dataset)
train_size = int(0.9 * total_size)  # 90% for training
val_size = total_size - train_size
# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for features, labels in train_dataset:
    print(features)
    print(len(features))
    print(features.device)
    break

In [ ]:
# Simple training loop
num_epochs = 25
train_losses, val_losses = [], []

model = BaseClassifier(input_size = 21)
model.to(Device)

# Loss function
criterion = nn.BCEWithLogitsLoss(reduction = 'sum')
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    all_targets = []
    all_preds = []
    for features, labels in tqdm(train_loader, desc='Training loop'):
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        preds = (outputs > 0.5).float()
        all_targets.extend(labels.to('cpu').numpy())
        all_preds.extend(preds.squeeze().detach().to('cpu').numpy())
        
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accuracy = accuracy_score(all_targets, all_preds)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    all_targets = []
    all_preds = []
    with torch.inference_mode():
        for features, labels in tqdm(val_loader, desc='Validation loop'):  
            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            running_loss += loss.item()
            
            preds = (outputs > 0.5).float()
            all_targets.extend(labels.to('cpu').numpy())
            all_preds.extend(preds.squeeze().detach().to('cpu').numpy())
            
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    test_accuracy = accuracy_score(all_targets, all_preds)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Train accuracy: {train_accuracy}, Validation loss: {val_loss}, Test accuracy: {test_accuracy}")

In [ ]:
test_dataset = spaceDataset(csv_file = '/kaggle/input/spaceship-titanic/test.csv', target_col = None, drop_col = drop_col, split_col = split_col, dummy_col = dummy_col, norm_col = norm_col, scaler = scaler, Device = Device)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
for features in test_dataset:
    print(features)
    print(len(features))
    print(features.device)
    break

In [ ]:
model.eval()
all_preds = []
with torch.inference_mode():
    for features in tqdm(test_loader, desc='Testing loop'):  
        outputs = model(features)

        preds = (outputs > 0.5).float()
        all_preds.extend(preds.squeeze().detach().to('cpu').numpy())

In [ ]:
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_df['Transported'] = np.array(all_preds, dtype = bool)
test_df[['PassengerId','Transported']].to_csv('output_2.csv', index = False)